In [1]:
import numpy as np
import os
import sys
import scipy
import cv2
import gc

#解析使用
import xml
from xml.etree import ElementTree as ET

from glob import glob

import keras.backend as K
from keras.applications import VGG19
from keras.models import Model
from keras.utils import to_categorical

import imageio
from skimage import transform

from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.svm import SVC #类别分类使用
from sklearn.linear_model import Ridge #bounding-box回归
from sklearn.externals import joblib

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf

from tensorflow.contrib import slim

from ImageNet_classes import class_names #验证alexnet使用

In [3]:
TRAIN_DATA_PATH = '../../../tensorflow2/dataset/VOCtrainval_11-May-2012/JPEGImages/'
TEST_DATA_PATH = '../../../tensorflow2/dataset/VOC2012test/JPEGImages/'

TRAIN_XML_PATH = '../../../tensorflow2/dataset/VOCtrainval_11-May-2012/Annotations/'
TEST_XML_PATH = '../../../tensorflow2/dataset/VOC2012test/Annotations/'

CLASSES_NUM = 20

STR = [
    'person',
    'bird','cat','cow','dog','horse','sheep',
    'aeroplane','bicycle','boat','bus','car','motorbike','train',
    'bottle','chair','diningtable','pottedplant','sofa','tvmonitor'
]

LABEL2STR = {idx:value for idx , value in enumerate(STR)}
STR2LABEL = {value:key for key,value in LABEL2STR.items()}
#STR2LABEL = {value:idx for idx , value in enumerate(STR)}

STR2LABEL['none'] = 'none' #先不使用part部分 只进行naive目标检测

#目标检测相关
IoU_THRESHOLD = 0.5

#SVM相关
SVM_IoU_THRESHOLD = 0.3

#NMS相关
NMS_IoU_THRESHOLD = 0.3 #or ~0.5

#bbox回归
BBOX_REGRESS_IoU_THRESHOLD = 0.6

In [4]:
xml_file_names_train = glob(TRAIN_XML_PATH + '*') #所有的xml文件 完整路径

#从xml文件中读出图片相关的信息

def xml_parse(xml_file):
    '''
    return filename , shape , name_boxes , crop_boxes
    xml文件中的shape格式为 (width height 3)
    '''
    xml_file = xml.dom.minidom.parse(xml_file)
    xml_file_docu_ele = xml_file.documentElement

    filename_list = xml_file_docu_ele.getElementsByTagName('filename')
    
    #filename_list可能有多个filename的 所以要索引0(此数据集中filename只有一个)
    filename = filename_list[0].childNodes[0].data #filename_list.firstChild.data

    #图像的尺寸信息
    size_list = xml_file_docu_ele.getElementsByTagName('size')

    for size in size_list:
        width_list = size.getElementsByTagName('width')
        width = int(width_list[0].childNodes[0].data)

        height_list = size.getElementsByTagName('height')
        height = int(height_list[0].childNodes[0].data)

        channel_list = size.getElementsByTagName('depth')
        channel = int(channel_list[0].childNodes[0].data)

    #一个文件中有多个object
    object_list = xml_file_docu_ele.getElementsByTagName('object')

    #多个object与多个object对应的详细信息
    name_boxes = [] #一个元素就是一个object
    crop_boxes = []

    for objects in object_list:
        #一次循环处理一个object信息
        #一个xml文件（即一个图像中）有多个object

        #name
        name_list = objects.getElementsByTagName('name')

        name_box = name_list[0].childNodes[0].data

        #bounding box points
        bndbox = objects.getElementsByTagName('bndbox')

        x1_list = bndbox[0].getElementsByTagName('xmin')
        x1 = int( round( float(x1_list[0].childNodes[0].data) ) )

        y1_list = bndbox[0].getElementsByTagName('ymin')
        y1 = int(round(float( y1_list[0].childNodes[0].data )))

        x2_list = bndbox[0].getElementsByTagName('xmax')
        x2 = int(round(float( x2_list[0].childNodes[0].data )))

        y2_list = bndbox[0].getElementsByTagName('ymax')
        y2 = int(round(float( y2_list[0].childNodes[0].data )))

        crop_box = [x1,x2,y1,y2]

        name_boxes.append(name_box)
        crop_boxes.append(crop_box)

    #crop_box:[x1 x2 y1 y2]
    return filename , name_boxes , np.array(crop_boxes) #filename调试使用


In [5]:
#xml_parse(xml_file_names_train[897])

In [6]:
class Image(object):
    '''
    图片的真实信息
    '''
    def __init__(self):
        self.img_file_names_train = glob(TRAIN_DATA_PATH+'*') #训练全路径信息
                
    def load(self , img_path_name = None):
        if not img_path_name:
            img_path_name = np.random.choice(self.img_file_names_train) #随机选择一张图片
            #img_path_idx = np.random.randint(0 , high = len(self.img_file_names_train)) #随机索引

        img_arr = cv2.imread(img_path_name) #BGR height*width*chanel
        
        xml_file_name = TRAIN_XML_PATH + img_path_name[-15:-4] +  '.xml'
        
        _ , name_boxes , crop_boxes = xml_parse(xml_file_name)
        
        labels = [] #存储与bndbox对应的 label信息

        for i in range(len(crop_boxes)): #多个object 
            labels.append(STR2LABEL.get(name_boxes[i] , 'none'))
        
        return img_arr , labels , crop_boxes
    

In [7]:
class Img_generator(object):
    def __init__(self):
        self.img_loader = Image()

    #计算bbox面积
    def bbox_area(self , bbox):
        w = bbox[1] - bbox[0]
        h = bbox[3] - bbox[2]
        
        return w*h
    
    #计算交并比
    def IoU(self , bbox_a , bbox_b):
        xmin_a = bbox_a[0]
        xmax_a = bbox_a[1]
        ymin_a = bbox_a[2]
        ymax_a = bbox_a[3]
        
        xmin_b = bbox_b[0]
        xmax_b = bbox_b[1]
        ymin_b = bbox_b[2]
        ymax_b = bbox_b[3]
        
        if   xmin_a < xmax_b <= xmax_a and (ymin_a < ymax_b <= ymax_a or ymin_a <= ymin_b < ymax_a):
            flag = True
        elif xmin_a <= xmin_b < xmax_a and (ymin_a < ymax_b <= ymax_a or ymin_a <= ymin_b < ymax_a):
            flag = True
        elif xmin_b < xmax_a <= xmax_b and (ymin_b < ymax_a <= ymax_b or ymin_b <= ymin_a < ymax_b):
            flag = True
        elif xmin_b <= xmin_a < xmax_b and (ymin_b < ymax_a <= ymax_b or ymin_b <= ymin_a < ymax_b):
            flag = True
        else:
            flag = False
        
        if flag:
            x_sorted_list = sorted([xmin_a, xmax_a, xmin_b, xmax_b])
            y_sorted_list = sorted([ymin_a, ymax_a, ymin_b, ymax_b])
            
            x_intersect_w = x_sorted_list[2] - x_sorted_list[1] #0 1 2 3
            y_intersect_h = y_sorted_list[2] - y_sorted_list[1] #0 1 2 3
            
            area_inter = x_intersect_w * y_intersect_h #计算重合面积
            
            union_area = self.bbox_area(bbox_a) + self.bbox_area(bbox_b) - area_inter
            
            return area_inter/union_area
        else:
            return 0.0
    
    
    def map2new(self , img_shape , ground_truth_coord):
        '''
        坐标系映射至448*448坐标系中
        '''
        original_height = img_shape[0]
        original_width = img_shape[1]
        
        ground_truth_coord[: , :2] = np.array( ground_truth_coord[: , :2] * (448/original_width) , dtype=int) #x1 x2
        ground_truth_coord[: , 2:] = np.array( ground_truth_coord[: , 2:] * (448/original_height) , dtype=int) #y1 y2
        
        return ground_truth_coord
        
    
    def get_train_proposal(self , labels , ground_truth_coord):
        #get_train_proposal为关键函数
        
        def _center(gt):
            '''
            gt的中心坐标
            '''
            x = int( ( gt[0] + gt[1] ) / 2 )
            y = int( ( gt[2] + gt[3] ) / 2 )
        
            return [x,y]
        
        def _is_in_grid(gt_center , grid):
            '''
            判断gt的中心是否在此grid cell中
            '''
            if (grid[0] <= gt_center[0] <= grid[1]) and (grid[2] <= gt_center[1] <= grid[3]):
                return True
            else:
                return False
        
        def _target_gt(gt , grid):
            '''
            将gt变为target需要的格式
            '''
            center = _center(gt)
            #gt中心坐标在对应的grid中的偏移
            target_x = ( center[0] - grid[0] ) / 64
            target_y = ( center[1] - grid[2] ) / 64
            
            target_w = ( gt[1]-gt[0] ) / 448
            target_h = ( gt[3]-gt[2] ) / 448
            
            return [target_x , target_y , target_w , target_h]
            
        
        '''下面操作在448*448坐标系中进行'''
        
        #S*S grid cells
        x_slice = [ [64*i , 64*(i+1)] for i in [0,1,2,3,4,5,6] ]
        y_slice = [ [64*i , 64*(i+1)] for i in [0,1,2,3,4,5,6] ]
        
        grid = np.zeros(shape=[7 , 7] , dtype=list)
        
        for x_idx , x in enumerate(x_slice):
            for y_idx , y in enumerate(y_slice):
                grid[x_idx][y_idx] = y + x
        
        #训练样本中的y
        #[confidence x y w h]*2 + cls_score
        target = np.zeros(shape=[7 , 7 , 30] , dtype=float)
        
        for i in range(7):
            for j in range(7):
                for idx , gt in enumerate(ground_truth_coord):
                    
                    if _is_in_grid( _center(gt) , grid[i][j]):
                        '''
                        此gt的中心位于此grid cell中
                        '''
                        iou = self.IoU(gt , grid[i][j])
                        
                        #如果出现一个grid cell有多个gt对应 则只保留iou最高的前两个
                        if target[i][j][0] > target[i][j][5]:
                            if iou > target[i][j][5]:
                                target[i][j][5] = iou
                                target[i][j][6 : 10] = _target_gt(gt , grid[i][j])
                                target[i][j][ labels[idx] + 10 ] = 1.0
                        elif target[i][j][0] <= target[i][j][5]:
                            if iou > target[i][j][0]:
                                target[i][j][0] = iou
                                target[i][j][1 : 5] = _target_gt(gt , grid[i][j])
                                target[i][j][ labels[idx] + 10 ] = 1.0
                        #并入上面
                        #else:
                        #    #先执行此处
                        #    if iou > target[i][j][0]:
                        #        #只要大于 随意选一个位置即可
                        #        target[i][j][0] = iou
                        #        target[i][j][1 : 5] = _target_gt(gt)
                        #        target[i][j][ labels[idx] + 10 ] = 1.0
                
                #here
                #处理完一个grid cell
                #如果[i , j] grid只有一个gt与之对应 则将其翻倍（因为yolo v1一个grid对应两个bounding box）
                #只有一个gt与之对应 则只会出现在target[i][j][0 1 2 3 4]处
                if (target[i][j][0] != 0.0) and (target[i][j][5] == 0.0):
                    target[i][j][5:10] = target[i][j][0:5]
                    #cls_score是一样的 同一个位置
                                                  
        return np.array(target)
    
    
    def load(self , img_path_name):
        '''
        img_path_name:绝对路径
        '''
        
        #图片数据 ground truth具体数据 ground truth对应label ground truth坐标信息 图片文件名
        img_arr , labels , ground_truth_coord = self.img_loader.load(img_path_name)
        
        ground_truth_coord = self.map2new(img_arr.shape , ground_truth_coord) #将ground_truch坐标从原坐标系映射至448*448坐标系中
        
        img_arr = cv2.resize(img_arr , (448 , 448))
        img_arr = img_arr / 127.5 - 1 #对下面的get_train_proposal没有影响
        
        target = self.get_train_proposal(labels , ground_truth_coord)
        
        '''
        resize 并 归一化像素值
        img_arr 为 BGR形式
        '''
        
        '''[R G B] [123.68 116.779 103.939]
        减去每个通道的像素平均值 归一化'''
        #img_arr[:,:,0] = img_arr[:,:,0] - 103.939
        #img_arr[:,:,1] = img_arr[:,:,1] - 116.779
        #img_arr[:,:,2] = img_arr[:,:,2] - 123.680
        
        #'''增加一维 batch维'''
        return np.expand_dims(img_arr , axis=0) , target
    
    
    def load_test(self , img_path_name):
        img_arr = cv2.imread(img_path_name)
        
        img_arr_resize = cv2.resize(img_arr , (448 , 448))
        img_arr_resize_norm = img_arr_resize / 127.5 - 1
        
        return np.expand_dims(img_arr_resize_norm , axis=0) , img_arr


In [8]:
class Dataset(object):
    def __init__(self):
        self.img_generator = Img_generator()
        
        self.img_loader = Image()
        
        self.img_file_names_train = glob(TRAIN_DATA_PATH + '*')
        self.img_file_names_test = glob(TEST_DATA_PATH + '*')
    
    def get_batch(self):
        path = np.random.choice(self.img_file_names_train)
        
        x , target = self.img_generator.load(path)
    
        return x , target
    
    def get_batch_test(self , path):
        
        if not path:
            #未指定path 从测试目录中随机选一张图片测试
            path = np.random.choice(self.img_file_names_test)
            print('test image:', path)
        
        
        x , img_arr= self.img_generator.load_test(path)
        
        return x , img_arr

In [9]:
#refer:https://blog.csdn.net/two_vv/article/details/76769860
#alexnet原始模型以及预训练参数导入
class AlexNet_model(object):
    def __init__(self , is_training=True):
        
        self.x = tf.placeholder(dtype=tf.float32 , shape=[1 , 448 , 448 , 3])        
        
        self.build(is_training) #构建网络产生输出
        
        if is_training:
            self.target = tf.placeholder(dtype=tf.float32 , shape=[7 , 7 , 30])
            self.loss()

    def build(self , is_training):
        #arch from paper
        def _conv(_input , num_outputs , kernel_size , stride=1):
            return slim.conv2d(_input , num_outputs=num_outputs , kernel_size=kernel_size , stride=stride , activation_fn=tf.nn.leaky_relu ,
                             weights_initializer=tf.initializers.truncated_normal(stddev=0.01) ,
                             biases_initializer=tf.initializers.constant(0.0))
        
        def _max_pool(_input , kernel_size=2 , stride=2):
            return slim.max_pool2d(_input , kernel_size=kernel_size , stride=stride)
        
        def _conv_module_a(_input):
            _output = _conv(_input , 256 , 1)
            return _conv(_output , 512 , 3)
        
        def _conv_module_b(_input):
            _output = _conv(_input , 512 , 1)
            return _conv(_output , 1024 , 3)
        
        output = _conv(self.x , 64 , 7 , 2)
        output = _max_pool(output)
        
        output = _conv(output , 192 , 3)              
        output = _max_pool(output)
        
        output = _conv(output , 128 , 1)
        output = _conv(output , 256 , 3)
        output = _conv(output , 256 , 1)
        output = _conv(output , 512 , 3)
        output = _max_pool(output)
        
        #4 times
        output = _conv_module_a(output)
        output = _conv_module_a(output)
        output = _conv_module_a(output)
        output = _conv_module_a(output)
        
        output = _conv(output , 512 , 1)
        output = _conv(output , 1024 , 3)
        output = _max_pool(output)
        
        #twice
        output = _conv_module_b(output)
        output = _conv_module_b(output)

        output = _conv(output , 1024 , 3)
        output = _conv(output , 1024 , 3 , 2)
        
        output = _conv(output , 1024 , 3)
        output = _conv(output , 1024 , 3)
        
        output = slim.flatten(output)
        
        #paper中为4096 pc性能达不到
        output = slim.fully_connected(inputs=output , num_outputs=1024 , activation_fn=tf.nn.leaky_relu ,
                                     weights_initializer=tf.initializers.truncated_normal(stddev=0.01),
                                     biases_initializer=tf.initializers.constant(0.0))
        
        #引入dropout
        output = slim.dropout(output , keep_prob=0.5 , is_training=is_training)
        
        #tf.identity 使用线性激活函数 nan错误 使用leaky relu也会出错 换成relu
        output = slim.fully_connected(inputs=output , num_outputs=7*7*30 , activation_fn=tf.nn.relu  ,
                                     weights_initializer=tf.initializers.truncated_normal(stddev=0.01),
                                     biases_initializer=tf.initializers.constant(0.0))
        
        self.output = tf.reshape(output , shape=[7 , 7 , 30]) #丢弃掉batch维 没用

        
    def loss(self):
        lambda_coord = 5.0
        lambda_noobj = 0.5
        
        #[:,:,0]
        _mask = tf.cast( tf.greater( tf.slice(self.target,begin=[0,0,0],size=[7,7,1]) , np.zeros(shape=[7,7,1] , dtype=float) ) , dtype=tf.float32 )
        mask = tf.tile(_mask , multiples=[1 , 1 , 2]) #7*7*2
        #[:,:,[1,2]]
        loss_coord = tf.reduce_sum( tf.square( ( tf.slice(self.output,begin=[0,0,1],size=[7,7,2]) - tf.slice(self.target,begin=[0,0,1],size=[7,7,2]) ) * mask ) )
        #[:,:,[3,4]]
        loss_coord += tf.reduce_sum( tf.square( (tf.sqrt( tf.slice(self.output,begin=[0,0,3],size=[7,7,2]) ) - tf.sqrt( tf.slice(self.target,begin=[0,0,3],size=[7,7,2]))) * mask ) )
        
        loss_coord = lambda_coord * loss_coord
        
        #[:,:,:[0,5]]
        loss_iou = tf.reduce_sum( tf.square( ( tf.slice(self.output,begin=[0,0,0],size=[7,7,1]) - tf.slice(self.target,begin=[0,0,0],size=[7,7,1])) * mask ) )
        loss_iou += tf.reduce_sum( tf.square( ( tf.slice(self.output,begin=[0,0,5],size=[7,7,1]) - tf.slice(self.target,begin=[0,0,5],size=[7,7,1])) * mask ) )
        #[:,:,:[0,5]]
        loss_iou += lambda_noobj * (tf.reduce_sum( tf.square( ( tf.slice(self.output,begin=[0,0,0],size=[7,7,1]) - tf.slice(self.target,begin=[0,0,0],size=[7,7,1]) ) * (1.0-mask) ) ) +\
                                    tf.reduce_sum( tf.square( ( tf.slice(self.output,begin=[0,0,5],size=[7,7,1]) - tf.slice(self.target,begin=[0,0,5],size=[7,7,1]) ) * (1.0-mask) ) ))
        
        mask = tf.tile(_mask , multiples=[1,1,20]) #7*7*20 (因为20 classes)
        #[:,:,10:]
        loss_cls = tf.reduce_sum( tf.square( ( tf.slice(self.output,begin=[0,0,10],size=[7,7,20]) - tf.slice(self.target,begin=[0,0,10],size=[7,7,20]) ) * mask ) )
        
        self.total_loss = loss_coord + loss_iou + loss_cls


In [10]:
def display(img_arr , labels , bbox , name):    
    
    print('debug:' , img_arr.shape , labels , bbox)
    
    for i in range(len(labels)):

        x1 = bbox[i][0]
        x2 = bbox[i][1]
        y1 = bbox[i][2]
        y2 = bbox[i][3]

        img_arr = cv2.rectangle(img_arr , (x1 , y1) , (x2 , y2) , (255,255,255))

        img_arr = cv2.putText(img_arr , LABEL2STR[labels[i]] , org=(x1 , y1+10) , fontFace = cv2.FONT_HERSHEY_PLAIN , fontScale=1 , color = (255,255,255), thickness = 1)

    #plt.imshow(meta_img) #图像查看

    plt.imsave(arr=img_arr[: , : ,[2,1,0]] , fname = 'result/%s.jpg' % name) #保存图像


In [11]:
#refer:https://blog.csdn.net/two_vv/article/details/76769860

class YOLO_V1(object):
    '''
    完整模型
    '''
    
    def __init__(self , is_training = True):
        self._grid() #创建grid cell 坐标信息
        
        self.dataset = Dataset()
        
        self.filewriter_path = 'save/logs' #模型可视化
        self.checkpoint_path = 'save/model/' #模型持久化
                              
        self.model = AlexNet_model(is_training)
        
        self.sess = tf.Session()
        
        self.saver = tf.train.Saver(max_to_keep=2) #max_to_keep 最大保存5次模型  之后继续保存则会覆盖前面的模型
        
        if is_training:
            '''训练参数'''
            self.epoch = 100000
            
            self.global_step = tf.Variable(initial_value=0 , trainable=False)
            
            self.learning_rate = tf.train.exponential_decay(learning_rate=0.00001 , global_step=self.global_step,
                                                            decay_steps=900 , decay_rate=0.8 , staircase=True)
            
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.model.total_loss , global_step=self.global_step)
        
            '''引入滑动平均'''
            self.ema = tf.train.ExponentialMovingAverage(decay=0.9) #滑动平均
            self.average_op = self.ema.apply(tf.trainable_variables()) #给所有的可训练变量应用滑动平均
            
            with tf.control_dependencies([self.optimizer]):
                self.train_op = tf.group(self.average_op)
            
            #初始化参数 与 持久化 可视化 相关
            self.sess.run(tf.global_variables_initializer())
            tf.summary.scalar('total_loss' , self.model.total_loss)
            self.merged_summary = tf.summary.merge_all() #merge all summaries in the default graph
            self.writer = tf.summary.FileWriter(self.filewriter_path , self.sess.graph) #可视化
            
        
    
    def _grid(self):
        #S*S grid cells
        x_slice = [ [64*i , 64*(i+1)] for i in [0,1,2,3,4,5,6] ]
        y_slice = [ [64*i , 64*(i+1)] for i in [0,1,2,3,4,5,6] ]
        
        self.grid = np.zeros(shape=[7 , 7] , dtype=list)
        
        for x_idx , x in enumerate(x_slice):
            for y_idx , y in enumerate(y_slice):
                self.grid[x_idx][y_idx] = y + x
        
    
    def train(self):
        
        if os.path.exists(self.checkpoint_path+'checkpoint'):
            self.saver.restore(self.sess , tf.train.latest_checkpoint(self.checkpoint_path))
        else:
            self.sess.run(tf.global_variables_initializer()) 
        
        for i in range(100000):
            x , target = self.dataset.get_batch()
            
            self.sess.run(self.train_op , feed_dict={self.model.x : x , self.model.target : target} )

            if i % 10 == 0:
                self.saver.save(self.sess , self.checkpoint_path + 'model.ckpt' , global_step = i)
                
                _output , total_loss , summary = self.sess.run([self.model.output , self.model.total_loss , self.merged_summary] , feed_dict={self.model.x : x , self.model.target : target})
                
                self.writer.add_summary(summary , global_step = i)
                
                print(i , total_loss , _output)
            
        self.writer.close() #event to disk and close the file

    def predict(self , path=None):
        if os.path.exists(self.checkpoint_path + 'checkpoint'):
            self.saver.restore(self.sess , tf.train.latest_checkpoint(self.checkpoint_path) )
            
            self._predict(path)
        ===================
        这里不导入预训练的参数 直接使用随机的参数进行尝试 看一下输出是否是不变的
        else:
            print('no model!!!')
            return 
    
    def _predict(self , path):
        threshold_1 = 0.001
        threshold_nms = 0.7
        
        x , img_arr= self.dataset.get_batch_test(path)
        
        output = self.sess.run(self.model.output , feed_dict={self.model.x : x})
        self._output = output
        
        prod = self._confidence_p(output)
        bbox = self._bbox(output)
                
        self._prod = prod
        self._bbox = bbox
        
        pred_labels = []
        pred_bnds = []
        
        #step 1 set zero
        prod = np.greater(prod , threshold_1).astype(dtype=int) * prod #小于阈值置0
        
        #step 2 sort and nms
        for i in range(20):
            des_idx = np.argsort(-1 * prod[i]) #降序排列
            
            nms_idx = self._nms(prod[i][des_idx] , bbox[: , des_idx] , des_idx , threshold_nms) #返回的对应被丢弃的bnd box索引
            
            #将丢弃的confidence*pi置0
            prod[i][nms_idx] = 0
        
            #step 3 final prediction
            if np.max(prod[i]) != 0:
                pred_idx = np.argmax(prod[i])
                                
                pred_labels.append(i)
                pred_bnds.append( self._to_original(bbox[: , pred_idx] , img_arr.shape) )
        
        #step 4 draw in image
        display(img_arr , pred_labels , pred_bnds , 'first')
        
    def _to_original(self , bbox , shape):
        return [int( bbox[0]*(shape[1]/448) ) , int( bbox[1]*(shape[1]/448) ) , #x1 x2
                 int( bbox[2]*(shape[0]/448) ) , int( bbox[3]*(shape[0]/448) ) ] #y1 y2
    
    def _confidence_p(self , output):
        #置信度与分类概率的乘积
        prod = []
        
        for i in range(7):
            for j in range(7):
                prod.append(output[i,j,0] * output[i,j, 10:]) #bnd 1
                prod.append(output[i,j,5] * output[i,j, 10:]) #bnd 2
                
        prod = np.array(prod) #98*20
        prod = prod.T #20*98
        
        return prod            
    
    def _bbox(self , output):
        #将output中的坐标还原为448*448坐标系中
        def __bbox(bnd , grid):
            target_x = bnd[0]
            target_y = bnd[1]
            target_w = bnd[2]
            target_h = bnd[3]
            
            center_x = int( target_x * 64 + grid[0] )
            center_y = int( target_y * 64 + grid[2] )
            
            w = int( target_w * 448 )
            h = int( target_h * 448 )
            
            x1 = int( center_x - 0.5*w )
            x2 = int( center_x + 0.5*w )
            y1 = int( center_y - 0.5*h )
            y2 = int( center_y + 0.5*h )
            
            return [x1,x2,y1,y2]
        
        bbox = []
        
        for i in range(7):
            for j in range(7):
                bbox.append( __bbox(output[i,j,1:5] , self.grid[i][j]) ) #bnd 1
                bbox.append( __bbox(output[i,j,6:10], self.grid[i][j]) ) #bnd 2
    
        bbox = np.array(bbox) #98*4
        bbox = bbox.T #4*98
        
        return bbox
        

    def _nms(self , prod , bbox , des_idx , threshold_nms):
        '''
        prod已降序排列 bbox也对应降序 des_idx为在未降序的数组中的 降序索引(argsort)
        '''
        length = len(prod) #98
        lost_flag = [1]*length #标记丢弃的框 0表示丢弃
        
        max_score_idx = 0 #记录当前最大score的idx
        
        #对于score为0的bnd box不必进行nms 直接丢弃
        for i in range(length):
            if prod[i] == 0.0:
                lost_flag[i:] = [0] * (length - i) #因为prod已经降序 故出现0.0 后续全为0.0
                break
        
        while max_score_idx < length:
            max_score_rect = bbox[max_score_idx]
            
            for i in range(max_score_idx+1 , length):
                if lost_flag[i] == 1 and self._iou( max_score_rect , bbox[i] ) > threshold_nms: #大于阈值 丢弃
                    lost_flag[i] = 0

            max_score_idx_bak = max_score_idx #后续使用
            
            #让max_score_idx指向下一个没被丢弃的最大值
            for i in range(max_score_idx+1 , length):
                if lost_flag[i] == 1:
                    max_score_idx = i
                    break
            
            #说明max_score_idx没有移动过 即后续的都被丢弃了 提前终止循环
            if max_score_idx == max_score_idx_bak:
                break
            
        nms_idx = [] #用来存放丢弃的bnd box!!!

        for i in range(length):
            if lost_flag[i] == 0:
                nms_idx.append( des_idx[i] )
                
        return nms_idx
    
    def _iou(self , bbox_a , bbox_b):
        #计算bbox面积
        def __area(bbox):
            w = bbox[1] - bbox[0]
            h = bbox[3] - bbox[2]

            return w*h
    
        xmin_a = bbox_a[0]
        xmax_a = bbox_a[1]
        ymin_a = bbox_a[2]
        ymax_a = bbox_a[3]
        
        xmin_b = bbox_b[0]
        xmax_b = bbox_b[1]
        ymin_b = bbox_b[2]
        ymax_b = bbox_b[3]
        
        if   xmin_a < xmax_b <= xmax_a and (ymin_a < ymax_b <= ymax_a or ymin_a <= ymin_b < ymax_a):
            flag = True
        elif xmin_a <= xmin_b < xmax_a and (ymin_a < ymax_b <= ymax_a or ymin_a <= ymin_b < ymax_a):
            flag = True
        elif xmin_b < xmax_a <= xmax_b and (ymin_b < ymax_a <= ymax_b or ymin_b <= ymin_a < ymax_b):
            flag = True
        elif xmin_b <= xmin_a < xmax_b and (ymin_b < ymax_a <= ymax_b or ymin_b <= ymin_a < ymax_b):
            flag = True
        else:
            flag = False
        
        if flag:
            x_sorted_list = sorted([xmin_a, xmax_a, xmin_b, xmax_b])
            y_sorted_list = sorted([ymin_a, ymax_a, ymin_b, ymax_b])
            
            x_intersect_w = x_sorted_list[2] - x_sorted_list[1] #0 1 2 3
            y_intersect_h = y_sorted_list[2] - y_sorted_list[1] #0 1 2 3
            
            area_inter = x_intersect_w * y_intersect_h #计算重合面积
            
            union_area = __area(bbox_a) + __area(bbox_b) - area_inter
            
            return area_inter/union_area
        else:
            return 0.0

In [12]:
test = YOLO_V1()

In [13]:
test.train()

INFO:tensorflow:Restoring parameters from save/model/model.ckpt-170
0 1.222476 [[[0.         0.46259436 0.68026036 ... 0.         0.         0.        ]
  [0.         0.45029384 0.6667075  ... 0.05810298 0.         0.        ]
  [0.         0.38495782 0.6343057  ... 0.         0.         0.        ]
  ...
  [0.         0.4303137  0.6116145  ... 0.         0.         0.        ]
  [0.         0.47145256 0.5922859  ... 0.         0.         0.        ]
  [0.         0.31549853 0.8006651  ... 0.         0.         0.        ]]

 [[0.         0.512785   0.616822   ... 0.         0.         0.        ]
  [0.         0.50342536 0.636136   ... 0.         0.         0.        ]
  [0.         0.4533476  0.51661867 ... 0.         0.         0.        ]
  ...
  [0.         0.35533157 0.41910923 ... 0.         0.         0.        ]
  [0.         0.47749206 0.55003846 ... 0.         0.         0.        ]
  [0.         0.39176068 0.46246165 ... 0.         0.         0.        ]]

 [[0.         0.6

30 1.6464168 [[[0.         0.45196626 0.6478999  ... 0.         0.         0.        ]
  [0.         0.42647055 0.67894375 ... 0.         0.         0.        ]
  [0.         0.42373264 0.64308536 ... 0.         0.         0.        ]
  ...
  [0.         0.47623482 0.66253823 ... 0.         0.         0.        ]
  [0.         0.3961356  0.5087361  ... 0.         0.         0.        ]
  [0.         0.31541815 0.74481076 ... 0.         0.         0.        ]]

 [[0.         0.46978006 0.5864798  ... 0.         0.         0.        ]
  [0.         0.48455012 0.6140609  ... 0.         0.         0.        ]
  [0.         0.46635732 0.59375226 ... 0.         0.         0.        ]
  ...
  [0.         0.4055296  0.5067429  ... 0.         0.         0.        ]
  [0.         0.457957   0.5544194  ... 0.         0.         0.        ]
  [0.         0.35531512 0.49705642 ... 0.         0.         0.        ]]

 [[0.         0.6237027  0.6072781  ... 0.         0.         0.        ]
  [0.    

60 4.2530394 [[[0.         0.39109185 0.63273716 ... 0.         0.         0.        ]
  [0.         0.46251985 0.65157354 ... 0.         0.         0.        ]
  [0.         0.4623256  0.64775723 ... 0.         0.         0.        ]
  ...
  [0.         0.47350496 0.67485374 ... 0.         0.         0.        ]
  [0.         0.39033237 0.56942016 ... 0.         0.         0.        ]
  [0.         0.40039062 0.78328294 ... 0.         0.         0.        ]]

 [[0.         0.5443718  0.6734975  ... 0.         0.         0.        ]
  [0.         0.4688951  0.64451057 ... 0.         0.         0.        ]
  [0.         0.5056584  0.5475286  ... 0.         0.         0.        ]
  ...
  [0.         0.380761   0.50850606 ... 0.         0.         0.        ]
  [0.         0.44558027 0.5174334  ... 0.         0.         0.        ]
  [0.         0.3602718  0.546597   ... 0.         0.         0.        ]]

 [[0.         0.6911858  0.5775821  ... 0.         0.         0.        ]
  [0.    

KeyboardInterrupt: 

In [13]:
test.sess.close()

In [51]:
tf.reset_default_graph()

In [47]:
testt = YOLO_V1(is_training=False)

In [48]:
testt.predict()

INFO:tensorflow:Restoring parameters from save/model/model.ckpt-170
test image: ../../../tensorflow2/dataset/VOC2012test/JPEGImages\2009_004014.jpg
debug: (367, 500, 3) [0] [[208, 225, 154, 159]]


In [45]:
testt._output[:,:,0]


array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [49]:
testt._output


array([[[0.        , 0.504704  , 0.68090415, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.5195909 , 0.7278229 , ..., 0.02176071,
         0.        , 0.        ],
        [0.        , 0.45599553, 0.7023109 , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.4431908 , 0.72183174, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.4301787 , 0.61158144, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.41390112, 0.81913537, ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.5107293 , 0.6913626 , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.5259184 , 0.6934573 , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.5319846 , 0.58507186, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.39969704, 0.52106553, ..., 0.        ,
         0.        , 0.        ],
        [0. 

In [40]:
testt._output


array([[[0.        , 0.50470406, 0.68090415, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.5195909 , 0.72782296, ..., 0.02176071,
         0.        , 0.        ],
        [0.        , 0.45599553, 0.7023109 , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.4431908 , 0.72183174, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.43017867, 0.61158144, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.41390115, 0.81913537, ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.5107293 , 0.6913626 , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.5259185 , 0.6934573 , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.5319846 , 0.5850719 , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.39969707, 0.5210656 , ..., 0.        ,
         0.        , 0.        ],
        [0. 